In [ ]:
import json
import requests
from folium import Map, TileLayer, GeoJson

endpoint = "http://0.0.0.0:8082"

In [ ]:
geojson = {
  "type": "Feature",
  "properties": {},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -113.8623046875,
          31.316101383495624
        ],
        [
          -79.1455078125,
          31.316101383495624
        ],
        [
          -79.1455078125,
          49.15296965617039
        ],
        [
          -113.8623046875,
          49.15296965617039
        ],
        [
          -113.8623046875,
          31.316101383495624
        ]
      ]
    ]
  }
}
bounds = (-113.8623046875, 31.316101383495624, -79.1455078125, 49.15296965617039)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=4
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)
m

### Create Mosaic for the whole collection

In [ ]:
# create Mosaic
body = {
    "collections": ["landsat-8-c2-l2"],
}

r = requests.post(
    f"{endpoint}/create",
    json=body,
    params={
        # Info to add to the tilejson response
        "minzoom": 8,
        "maxzoom": 12,
        "bounds": ",".join(map(str, bounds)),
        # query parameter to add to the tile URL
        "assets": "SR_B1",
        "rescale": "0,20000",
    }
).json()
print(r)

In [ ]:
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=7
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    attr="Mosaic",
    min_zoom=8,
    max_zoom=12,
    max_native_zoom=12,    
)
aod_layer.add_to(m)
m

### With Date and cloud-cover filter

In [ ]:
# create Mosaic
body = {
    "collections": ["landsat-8-c2-l2"],
    "datetime": "2018-01-01T00:00:00Z/2018-02-01T23:59:59Z",
    "query": {"eo:cloud_cover": {"gt": 75}}
}

r = requests.post(
    f"{endpoint}/create",
    json=body,
    params={
        # Info to add to the tilejson response
        "minzoom": 8,
        "maxzoom": 12,
        "bounds": ",".join(map(str, bounds)),
        # query parameter to add to the tile URL
        "assets": "SR_B1",
        "rescale": "0,20000",
    }
).json()
print(r)

In [ ]:
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=7
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    attr="Mosaic",
    min_zoom=8,
    max_zoom=12,
    max_native_zoom=12,
)
aod_layer.add_to(m)
m